In [12]:
from IPython.display import clear_output

import re
import time
import datetime

import requests
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

from tinydb import TinyDB, Query

In [2]:
db = TinyDB('data/custom_data/oddsportal_output.json')

In [3]:
tournaments = db.table('tournaments')
matches = db.table('matches')

In [4]:
query = Query()

In [5]:
"""
UPDATE THESE
"""
chrome_executable_path = r"C:\Users\Cristian Simionescu\Desktop\Master\Random\webdrivers\chromedriver.exe"
output_file_path = r"data/custom_data/oddsportal_output.txt"

In [6]:
base_url = "https://www.oddsportal.com"
start_url = "https://www.oddsportal.com/tennis/results/"
headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Mobile Safari/537.36"
}

In [11]:
with requests.get(start_url, headers=headers) as r:
    links = re.findall(b'foo="f"\s*href="([^"]+)', r.content)
links = list(map(lambda x: x.decode(), links))

c = Chrome(executable_path=chrome_executable_path)

START_INDEX = 6 #0
END_INDEX = len(links)
for index, link in enumerate(links[START_INDEX:END_INDEX]):
    clear_output(wait=True)
    print("Tournament ID",index+START_INDEX)
    full_link = base_url + link
    link = link.lstrip('/').replace('tennis/', '')
    sep_index = link.find('/')
    if sep_index < 0:
        continue  # not found

    country = link[:sep_index]
    c.get(full_link)
    tournament_name = c.find_element_by_xpath('//h1').get_attribute('textContent')
    surface = re.search('\(\s*([a-z]+)\s*\)', tournament_name)
    if surface:
        surface = surface.group(1)
    else:
        surface = ""
    tournament_name = tournament_name.split(surface)[0].replace("(","").strip()
    
    tournmanet_year_links = c.find_element_by_css_selector("#col-content > div.main-menu2.main-menu-gray > ul.main-filter").find_elements_by_css_selector("li > span > strong > a")
    tournmanet_year_links = [e.get_attribute("href") for e in tournmanet_year_links]
    for tournmanet_year_link in tournmanet_year_links:
        c.get(tournmanet_year_link)

        tournament_year = c.find_element_by_css_selector("#col-content > div.main-menu2.main-menu-gray > ul.main-filter > li > span.active").get_attribute("textContent").strip()
        table = c.find_element_by_id('tournamentTable')
        table = table.find_element_by_xpath('.//tbody')
        match_date = None
        # print(table.get_attribute("innerHTML"))
        for child in table.find_elements_by_xpath('.//tr'):
            if child.get_attribute('class') == "dark center":
                pass
            elif "nob-border" in child.get_attribute('class'):
                pass #match_date = child.find_element_by_xpath('.//th/span').get_attribute('textContent').strip()
            elif 'deactivate' in child.get_attribute('class'):
                match_time = players = final_score = info_val = ""
                odds = list()
                match_link = child.find_element_by_css_selector('td > a').get_attribute("href")
                
                c.get(match_link)
                
                players = c.find_element_by_css_selector("#col-content > h1").get_attribute("textContent")
                players = re.sub('<[^>]+>', '', players).split('-')
                
                match_time = c.find_element_by_css_selector("#col-content > p.date.datet").get_attribute("textContent")
                match_time = datetime.datetime.strptime(match_time.replace("  "," "),'%A, %d %b %Y, %H:%M')

                result = c.find_element_by_css_selector("#event-status > p").get_attribute("textContent").replace("Final result").strip()
                result = result.split(" ",1)
                score = result[0]
                sets = result[1][1:-1].split(", ")
                
                odds_table = c.find_element_by_css_selector("#odds-data-table")
                
                c.back()
                
#                 for kid in child.find_elements_by_xpath('.//td'):
#                     if "table-time" in kid.get_attribute('class'):
#                         match_time = kid.get_attribute('textContent')
#                     elif "table-participant" in kid.get_attribute('class'):
#                         players = kid.get_attribute("textContent")
#                         players = re.sub('<[^>]+>', '', players).split('-')
#                         match_link = kid.find_element_by_css_selector("a").get_attribute("href")
#                     elif "table-odds" in kid.get_attribute('class'):
#                         final_score = kid.get_attribute('textContent')
#                     elif "odds-nowrp" in kid.get_attribute('class'):
#                         odds.append(kid.get_attribute('textContent'))
                print("country:",country)
                print("tournament_name:",tournament_name)
                print("Surface:",surface)
                print("Tournament_year:",tournament_year)
                print("match_date:",match_date)
                print("match_time:",match_time)
                print("players:",players)
                print("score:",score)
                print("sets:",sets)
                print("odds:",odds)
                print("\n\n")
c.close()

Tournament ID 9
Country: argentina
Title: Buenos Aires 2 Challenger Men
Surface: clay
Tournament_year: 2018
Tournmanet_year_link: https://www.oddsportal.com/tennis/argentina/buenos-aires-2-challenger-men/results/
Current_data: 18 Nov 2018
Table_time: 16:45
Players: ['Cachin P. ', ' Andujar', 'Alba P.']
Match_link: https://www.oddsportal.com/tennis/argentina/buenos-aires-2-challenger-men/cachin-pedro-andujar-alba-pablo-fZorclnr/
Table_odds: 0:2
Odds_nowrp: ['3.12', '1.35']
Info_val: 11



Country: argentina
Title: Buenos Aires 2 Challenger Men
Surface: clay
Tournament_year: 2018
Tournmanet_year_link: https://www.oddsportal.com/tennis/argentina/buenos-aires-2-challenger-men/results/
Current_data: 17 Nov 2018
Table_time: 19:55
Players: ['Kuzmanov D. ', ' Cachin P.']
Match_link: https://www.oddsportal.com/tennis/argentina/buenos-aires-2-challenger-men/kuzmanov-dimitar-cachin-pedro-lhaakKwr/
Table_odds: 0:2
Odds_nowrp: ['1.61', '2.29']
Info_val: 11



Country: argentina
Title: Buenos Aires 

KeyboardInterrupt: 